In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from peakweather.dataset import PeakWeatherDataset

from src.arima_utils import (
    get_arima_dataset,
    plot_pandas_series,
    pacf_plot,
    test_arima_model,
    plot_results,
)

In [ ]:
ds = PeakWeatherDataset(
    root=None,
    compute_uv=False,
    station_type="meteo_station",
    freq="h",
    aggregation_methods={"temperature": "mean"},
)

### Sceglia quale stazione predire
Seleziona la tua stazione preferita dal dataset PeakWeatherDataset.
Se non le ricordi, puoi vedere l'elenco delle stazioni disponibili con `print(ds.stations)`.

In [ ]:
station = "???"

### Recupera i dati di training e test
Usa la funzione `get_arima_dataset` per ottenere i dati di training e test per la stazione selezionata.
Visualizza i dati di training con la funzione `plot_pandas_series`.

#### Domanda
Cosa noti riguardo l'andamento della serie temporale?

In [ ]:
train_series, test_series = get_arima_dataset(dataset=ds, station="???")

plot_pandas_series(series="???")

### Non-stazionarietà
Il valore medio della serie temporale sembra variare nel tempo, indicando una possibile non-stazionarietà.

Per rendere la serie stazionaria, possiamo applicare una differenziazione (differencing).
Questo comporta sottrarre, a ogni punto della serie, il valore del punto precedente.

#### Esercizio
Applica la differenziazione alla serie di training e visualizza il risultato con `plot_pandas_series`.

**Consigli:**
- il ciclo `for` ha la seguente struttura:
   ```python
   for i in range(start, stop, step):
       # corpo del ciclo
   ```
- puoi iterare dal primo all'ultimo elemento con `range(1, len(series_name))`.
- oppure dall'ultimo al primo con `range(len(series_name) - 1, 0, -1)`.
- per accedere/modificare il valore di un elemento in una `pandas.Series`, usa `series_name.iloc[i]`.
- puoi effettuare la sottrazione tra due elementi semplicemente con l'operatore `-`.
- esempio: `series_name.iloc[10] = series_name.iloc[10] - series_name.iloc[9]` assegna alla posizione 10 della serie la differenza tra il valore in posizione 10 e quello in posizione 9.

#### Domanda
Cosa noti riguardo l'andamento della serie differenziata?

In [ ]:
# Creiamo una copia dei dati di training per la differenziazione
diff_train_series = train_series.copy()

# Completa il ciclo per calcolare la differenziazione
for i in "???":
    diff_train_series.iloc[i] = "???"

# Rimuoviamo il primo elemento perchè non ha un valore precedente
diff_train_series = diff_train_series.iloc[1:]

# Visualizziamo la serie differenziata
plot_pandas_series(series="???")

### Scelta dei parametri p, d, q
Ora che abbiamo una serie stazionaria, possiamo analizzare la funzione di auto-correlazione parziale (PACF) per scegliere i parametri del modello SARIMA.

#### Esercizio
Plotta la funzione PACF della serie differenziata usando la funzione `pacf_plot`.

**Interpretazione:**
- Se la PACF diventa piccola dopo k istanti considera p=k.
- Se la PACF mostra un picco dopo K istanti considera P=K.

In [ ]:
pacf_plot(series=diff_train_series)

#### Esercizio
Scegli i valori dei parametri.

**Parametri:**
- p: ordine della parte autoregressiva (AR) - guarda la PACF
- d: grado di differenziazione (differencing) - abbiamo usato 1
- q: ordine della parte media mobile (MA) - la ignoriamo per semplicità
- P: ordine stagionale della parte autoregressiva (SAR) - guarda la PACF stagionale
- D: grado di differenziazione stagionale (differencing stagionale) - la ignoriamo per semplicità
- Q: ordine stagionale della parte media mobile (SMA) - la ignoriamo per semplicità
- s: periodicità stagionale - guarda il pico stagionale nella PACF

In [ ]:
# Teniamo q, D, Q a 0 per semplicità
q, D, Q = 0, 0, 0

# Imposta i valori di p, d, P, s in base alla tua analisi
p = "???"
d = "???"
P = "???"
s = "???"

### Addestramento del modello SARIMA
Ora possiamo addestrare il modello SARIMA con i parametri scelti.
Usiamo la classe `SARIMAX` di `statsmodels.tsa.statespace.sarimax`, che si occuperà di addestrare il modello tramite il metodo `.fit()`.

In [ ]:
print(f"Addestrando SARIMA({p},{d},{q})x({P},{D},{Q})_{s} ...")
model = SARIMAX(endog=train_series, order=(p, d, q), seasonal_order=(P, D, Q, s)).fit(
    disp=False
)
print(model.summary())

### Testiamo il modello appreso
Ora possiamo testare il modello SARIMA appreso sui dati di test.
Usiamo la funzione `test_arima_model` per calcolare l'errore medio assoluto (MAE) e ottenere le previsioni.

Usiamo un orizzonte di previsione di 24 ore (h=24).

In [ ]:
mae, results = test_arima_model(model=model, h="???", series=test_series)
print(f"Mean MAE over test set: {mae:.4f}")

### Visualizziamo alcune previsioni
Usiamo la funzione `plot_results` per visualizzare alcune previsioni del modello SARIMA.
Possiamo cambiare il numero di campioni da visualizzare modificando il parametro `num_samples`.

In [ ]:
plot_results(results=results, num_samples=5)